In [1]:
import requests
import random
import json

In [2]:
base_url = "https://ndear.xiv.in/registry"
keycloak_url = "https://ndear.xiv.in"

In [3]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

In [4]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [5]:
swaggerJson = resp.json()
swaggerJson["paths"].keys()

dict_keys(['/api/v1/Institute/{entityId}', '/api/docs/Institute.json', '/api/v1/Institute/sign', '/api/v1/Institute/search', '/api/v1/Institute/invite', '/api/v1/Institute', '/api/v1/Institute/{entityId}/{property}/{propertyId}', '/api/v1/Institute/{entityId}/{property}', '/api/v1/Institute/{entityId}/{property}/{propertyId}/send', '/api/v1/Teacher/{entityId}', '/api/docs/Teacher.json', '/api/v1/Teacher/sign', '/api/v1/Teacher/search', '/api/v1/Teacher/invite', '/api/v1/Teacher', '/api/v1/Teacher/{entityId}/{property}/{propertyId}', '/api/v1/Teacher/{entityId}/{property}', '/api/v1/Teacher/{entityId}/{property}/{propertyId}/send', '/api/v1/BaseAttestationField/{entityId}', '/api/docs/BaseAttestationField.json', '/api/v1/BaseAttestationField/sign', '/api/v1/BaseAttestationField/search', '/api/v1/BaseAttestationField/invite', '/api/v1/BaseAttestationField', '/api/v1/BaseAttestationField/{entityId}/{property}/{propertyId}', '/api/v1/BaseAttestationField/{entityId}/{property}', '/api/v1/Ba

In [31]:
jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

'/api/docs/Institute.json'

In [32]:
resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200

In [33]:
resp.json()

{'Institute': {'$id': '#/properties/Institute',
  'type': 'object',
  'title': 'The Institute Schema',
  'required': [],
  'properties': {'instituteName': {'$id': '#/properties/instituteName',
    'type': 'string',
    'title': 'Institute Name'},
   'address': {'type': 'object',
    'required': [],
    'properties': {'plot': {'type': 'string', 'title': 'Plot'},
     'street': {'type': 'string', 'title': 'Street'},
     'landmark': {'type': 'string', 'title': 'Landmark'},
     'locality': {'type': 'string', 'title': 'Locality'},
     'state': {'type': 'string', 'title': 'State'},
     'district': {'type': 'string', 'title': 'District'},
     'village': {'type': 'string', 'title': 'Village/Town/City'},
     'pincode': {'type': 'string', 'title': 'Pincode'}}},
   'establishmentYear': {'$id': '#/properties/establishmentYear',
    'type': 'string',
    'title': 'Year of establishment of institute'},
   'gstnId': {'$id': '#/properties/gstnId',
    'type': 'string',
    'title': 'GSTN ID'},
 

In [34]:
[f for f in swaggerJson["paths"].keys() if "/invite" in f]

['/api/v1/Institute/invite',
 '/api/v1/Teacher/invite',
 '/api/v1/BaseAttestationField/invite',
 '/api/v1/Student/invite',
 '/api/v1/Common/invite']

In [35]:
inviteUrl = [f for f in swaggerJson["paths"].keys() if "/invite" in f][1]
inviteUrl

'/api/v1/Teacher/invite'

In [36]:
def entity_for(userId):
    return  {"personalDetails":{
         "userName":userId,
         "dob":"2020-12-12",
     },
      "identityDetails":{"type":"id","value":userId}, 
      "contactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}}
     }

In [37]:
userId=12
json.dumps(entity_for(userId))

'{"personalDetails": {"userName": 12, "dob": "2020-12-12"}, "identityDetails": {"type": "id", "value": 12}, "contactDetails": {"mobile": "12", "email": "12@example.com", "address": {"street": "abc", "state": "Karnataka", "pincode": "560001"}}}'

# invite

In [38]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, inviteUrl), json=entity_for(userId)
)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
idx = resp.json()["result"][entity_name]["osid"]

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1645085037352,"params":{"resmsgid":"","msgid":"c302a2df-dcda-4781-aa0e-66f1b16b9ca7","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-d416775c-731c-40bc-9e69-20ebcff327e5"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1645085037352, 'params': {'resmsgid': '', 'msgid': 'c302a2df-dcda-4781-aa0e-66f1b16b9ca7', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-d416775c-731c-40bc-9e69-20ebcff327e5'}}}


In [39]:
idx

'1-d416775c-731c-40bc-9e69-20ebcff327e5'

In [40]:
userId

'47266833823'

In [41]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}

data = {
  'client_id': 'registry-frontend',
  'username': userId,
  'password': 'abcd@123',
  'grant_type': 'password'
}

response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloak_url), headers=headers, data=data)
print(response.json())
print(response.status_code)
token = response.json()["access_token"]


{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJVVG9zZ2JYUFZHdmk2a19tVnVEa1oxTTZHeGNtYnBDZW9lTHJBUjJKcC04In0.eyJleHAiOjE2NDUwODU2NDIsImlhdCI6MTY0NTA4NTA0MiwianRpIjoiNzE0ZDA2NDItNTg5My00MDY5LWJmY2MtMDE0NTRmZWM2M2JjIiwiaXNzIjoiaHR0cDovL25kZWFyLnhpdi5pbi9hdXRoL3JlYWxtcy9zdW5iaXJkLXJjIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjFjMTQxZWI0LWQwMTctNDFkNS05ODEwLTgxNmE1ZjZkZGVkZSIsInR5cCI6IkJlYXJlciIsImF6cCI6InJlZ2lzdHJ5LWZyb250ZW5kIiwic2Vzc2lvbl9zdGF0ZSI6ImI0ZTBkZTc5LWFjMGYtNGQ2ZS1hMjQzLWQxMzk4MDgwNzdmMSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly9sb2NhbGhvc3Q6NDIwMiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMiIsImh0dHBzOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwczovL25kZWFyLnhpdi5pbiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHA6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vMjAuMTk4LjY0LjEyOCJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtbmRlYXIiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aW

In [42]:
token


'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJVVG9zZ2JYUFZHdmk2a19tVnVEa1oxTTZHeGNtYnBDZW9lTHJBUjJKcC04In0.eyJleHAiOjE2NDUwODU2NDIsImlhdCI6MTY0NTA4NTA0MiwianRpIjoiNzE0ZDA2NDItNTg5My00MDY5LWJmY2MtMDE0NTRmZWM2M2JjIiwiaXNzIjoiaHR0cDovL25kZWFyLnhpdi5pbi9hdXRoL3JlYWxtcy9zdW5iaXJkLXJjIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjFjMTQxZWI0LWQwMTctNDFkNS05ODEwLTgxNmE1ZjZkZGVkZSIsInR5cCI6IkJlYXJlciIsImF6cCI6InJlZ2lzdHJ5LWZyb250ZW5kIiwic2Vzc2lvbl9zdGF0ZSI6ImI0ZTBkZTc5LWFjMGYtNGQ2ZS1hMjQzLWQxMzk4MDgwNzdmMSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly9sb2NhbGhvc3Q6NDIwMiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMiIsImh0dHBzOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwczovL25kZWFyLnhpdi5pbiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHA6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vMjAuMTk4LjY0LjEyOCJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtbmRlYXIiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX1

In [43]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

resp = requests.get("%s/api/docs/swagger.json"%base_url, headers=headers)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [201]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%(entity_name, idx), headers=headers)
print(response.status_code)
obj = response.json()
response.status_code, response.json()

200


(200,
 {'osUpdatedAt': '2022-02-15T08:34:33.092Z',
  'osCreatedAt': '2022-02-15T08:34:33.092Z',
  'osUpdatedBy': '',
  'contactDetails': {'osid': '1-bf6b46ae-5198-4ef4-a009-0c75aa01ae1c',
   'osUpdatedAt': '2022-02-15T08:34:33.092Z',
   'address': {'osid': '1-6c2ddf68-9ffe-4a07-a19c-13e43d1b01b1',
    'pincode': '560001',
    'osUpdatedAt': '2022-02-15T08:34:33.092Z',
    'osCreatedAt': '2022-02-15T08:34:33.092Z',
    'osUpdatedBy': '',
    'street': 'abc',
    'osCreatedBy': '',
    'state': 'Karnataka'},
   'osCreatedAt': '2022-02-15T08:34:33.092Z',
   'osUpdatedBy': '',
   'mobile': '97767099632',
   'osCreatedBy': '',
   'email': '97767099632@example.com'},
  'personalDetails': {'osid': '1-051191d3-c3b8-470e-873e-329b0f861168',
   'osUpdatedAt': '2022-02-15T08:34:33.092Z',
   'osCreatedAt': '2022-02-15T08:34:33.092Z',
   'osUpdatedBy': '',
   'dob': '2020-12-12',
   'osCreatedBy': '',
   'userName': '97767099632'},
  'osCreatedBy': '',
  'osid': '1-32000beb-ede1-47ab-b5d2-eb15587ce

## update name and experience

In [202]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = '{"contactDetails":{"name": "test name %s"}, "experience":[{"institute":"Some other school"}]}'%userId
print(data)
response = requests.put('http://localhost:8081/api/v1/Teacher/%s'%idx, headers=headers, data=data)

{"contactDetails":{"name": "test name 97767099632"}, "experience":[{"institute":"Some other school"}]}


In [203]:
response.status_code, response.json()

(200,
 {'id': 'sunbird-rc.registry.update',
  'ver': '1.0',
  'ets': 1644914092489,
  'params': {'resmsgid': '',
   'msgid': 'a74cac73-61cb-426f-b3b0-1b5e0532955d',
   'err': '',
   'status': 'SUCCESSFUL',
   'errmsg': ''},
  'responseCode': 'OK'})

In [204]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%(entity_name, idx), headers=headers)
assert response.status_code == 200
print(response.json())
assert response.json()["contactDetails"]["name"] == "test name %s"%userId
response.json()

{'osUpdatedAt': '2022-02-15T08:34:53.526Z', 'osCreatedAt': '2022-02-15T08:34:33.092Z', 'osUpdatedBy': 'osrc', 'contactDetails': {'osid': '1-3813c4d3-2cf4-4d2b-af57-6df58a7f1acf', 'osUpdatedAt': '2022-02-15T08:34:53.526Z', 'osCreatedAt': '2022-02-15T08:34:33.092Z', 'osUpdatedBy': 'osrc', 'name': 'test name 97767099632', 'mobile': '97767099632', 'osCreatedBy': '', 'email': '97767099632@example.com'}, 'personalDetails': {'osid': '1-051191d3-c3b8-470e-873e-329b0f861168', 'osUpdatedAt': '2022-02-15T08:34:53.526Z', 'osCreatedAt': '2022-02-15T08:34:33.092Z', 'osUpdatedBy': 'osrc', 'dob': '2020-12-12', 'osCreatedBy': '', 'userName': '97767099632'}, 'osCreatedBy': '', 'experience': [{'osUpdatedAt': '2022-02-15T08:34:53.526Z', 'osUpdatedBy': 'osrc', 'institute': 'Some other school', 'osid': '1-fd1c9192-4134-4b85-a013-b8b5f8aa887f'}], 'osid': '1-32000beb-ede1-47ab-b5d2-eb15587ce264', 'osOwner': ['135abd72-24d5-4098-b8fc-280f1ba80097'], 'identityDetails': {'osid': '1-29beb1ab-f240-4c6d-a679-9fe675

{'osUpdatedAt': '2022-02-15T08:34:53.526Z',
 'osCreatedAt': '2022-02-15T08:34:33.092Z',
 'osUpdatedBy': 'osrc',
 'contactDetails': {'osid': '1-3813c4d3-2cf4-4d2b-af57-6df58a7f1acf',
  'osUpdatedAt': '2022-02-15T08:34:53.526Z',
  'osCreatedAt': '2022-02-15T08:34:33.092Z',
  'osUpdatedBy': 'osrc',
  'name': 'test name 97767099632',
  'mobile': '97767099632',
  'osCreatedBy': '',
  'email': '97767099632@example.com'},
 'personalDetails': {'osid': '1-051191d3-c3b8-470e-873e-329b0f861168',
  'osUpdatedAt': '2022-02-15T08:34:53.526Z',
  'osCreatedAt': '2022-02-15T08:34:33.092Z',
  'osUpdatedBy': 'osrc',
  'dob': '2020-12-12',
  'osCreatedBy': '',
  'userName': '97767099632'},
 'osCreatedBy': '',
 'experience': [{'osUpdatedAt': '2022-02-15T08:34:53.526Z',
   'osUpdatedBy': 'osrc',
   'institute': 'Some other school',
   'osid': '1-fd1c9192-4134-4b85-a013-b8b5f8aa887f'}],
 'osid': '1-32000beb-ede1-47ab-b5d2-eb15587ce264',
 'osOwner': ['135abd72-24d5-4098-b8fc-280f1ba80097'],
 'identityDetails'

## invite a student

In [212]:
userId

'97767099632'

In [214]:
headers = {
    'content-type': 'application/json',
}

data = {"identityDetails":{
        "fullName":"Test %s"%userId, 
        "gender":"Male"
        },
        "educationDetails":[{
            "institute":"Some other school"
        }
        ]
       }
#{"name": "Prashant Joshi",\n\t\t"phoneNumber": "9876543210",\n\t\t"email": "prashant@upps.in",\n\t\t"school": "UP Public School"\n\t}


response = requests.post('http://localhost:8081/api/v1/Student/invite', headers=headers, json=data)


In [215]:
response.json()



{'id': 'sunbird-rc.registry.invite',
 'ver': '1.0',
 'ets': 1644915265621,
 'params': {'resmsgid': '',
  'msgid': 'fccebdf2-f44a-4db3-a296-52f2c1eeaec1',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'Student': {'osid': '1-f487ba74-2b5b-4d4b-9619-471e75aec617'}}}

In [216]:
sid = response.json()["result"]["Student"]["osid"]


In [217]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()


200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-45cecd72-ded5-4540-b3c8-861628537143'}],
 'osid': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
 'identityDetails': {'osid': '1-ea45774c-cd00-4bab-8c82-f0bf361c4990',
  'gender': 'Male',
  'fullName': 'Test 97767099632'}}

In [218]:
sid

'1-f487ba74-2b5b-4d4b-9619-471e75aec617'

## claim

In [219]:

headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

params = (
    ('send', 'true'),
)

data = {
    "entityName":"Student",
    "entityId":"%s"%sid,
    "name":"studentInstituteAttest",
    "additionalInput":{"idx": "%s"%sid}
}
response = requests.put('http://localhost:8081/api/v1/send', headers=headers, params=params, json=data)
print(response.status_code)
response.json()



200


{'id': 'sunbird-rc.registry.send',
 'ver': '1.0',
 'ets': 1644915273377,
 'params': {'resmsgid': '',
  'msgid': 'f398ba1a-139f-4422-8edd-8994f23534db',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'attestationOSID': '1-12c772ad-6dcc-412b-b61c-318ded5a0961'}}

In [220]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()

200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-45cecd72-ded5-4540-b3c8-861628537143'}],
 'studentInstituteAttest': [{'additionalInput': {'osid': '1-f8f46a25-8628-4e7e-94fa-2737018ea046',
    'idx': '1-f487ba74-2b5b-4d4b-9619-471e75aec617'},
   'entityName': 'Student',
   '_osState': 'ATTESTATION_REQUESTED',
   'name': 'studentInstituteAttest',
   'entityId': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
   'osid': '1-12c772ad-6dcc-412b-b61c-318ded5a0961',
   '_osClaimId': '96852340-a171-43b6-b357-d8666aafac60',
   'propertyData': '{"name":"Test 97767099632","institute":"Some other school"}'}],
 'osid': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
 'identityDetails': {'osid': '1-ea45774c-cd00-4bab-8c82-f0bf361c4990',
  'gender': 'Male',
  'fullName': 'Test 97767099632'}}

## As a Teacher get claims

In [221]:
token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJSQ19zUTBRVUJydVozbTFuTEFUaHFMUXFKcWpNN2g5ZGV2dnR2RVF6OEZFIn0.eyJleHAiOjE2NDQ5MTU4NjAsImlhdCI6MTY0NDkxNTI2MCwianRpIjoiNGUwMmViYWEtNTFkOS00OGNiLTlhMTgtNTA3MGJlZjJlNTVjIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIxMzVhYmQ3Mi0yNGQ1LTQwOTgtYjhmYy0yODBmMWJhODAwOTciLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJkMjdjN2FkYy04MzVhLTRhYWMtOTgzNS03MzMxOGE0NDQ0NjIiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

In [222]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response)
print(response.status_code, response.json())
claim_id = response.json()[-1]["id"]
response.json()

<Response [200]>
200 [{'id': '7e912173-329a-490f-9561-c9943ce6f693', 'entity': 'Student', 'entityId': '1-383ecafc-eea0-4bcc-a061-42336753ae81', 'propertyURI': '', 'createdAt': '2022-02-15T12:54:51.731+00:00', 'attestedOn': '2022-02-15T12:57:40.834+00:00', 'status': 'CLOSED', 'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))", 'attestorEntity': 'Teacher', 'requestorName': None, 'propertyData': '{"name":"Test 81540432738","institute":"Some other school"}', 'attestationId': '1-524c50f7-7241-4c7d-966c-a4dc1089c94f', 'attestationName': 'studentInstituteAttest', 'closed': True}, {'id': '325d7a14-900e-45d8-a928-82c92461969c', 'entity': 'Student', 'entityId': '1-8333e64a-4f4e-4d98-95b3-c4753f47acba', 'propertyURI': '', 'createdAt': '2022-02-15T07:57:08.877+00:00', 'attestedOn': None, 'status': 'OPEN', 'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))", 'attestorEntity': 'Teacher', 'requestorName': None, 'propertyData': '{"name

[{'id': '7e912173-329a-490f-9561-c9943ce6f693',
  'entity': 'Student',
  'entityId': '1-383ecafc-eea0-4bcc-a061-42336753ae81',
  'propertyURI': '',
  'createdAt': '2022-02-15T12:54:51.731+00:00',
  'attestedOn': '2022-02-15T12:57:40.834+00:00',
  'status': 'CLOSED',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Test 81540432738","institute":"Some other school"}',
  'attestationId': '1-524c50f7-7241-4c7d-966c-a4dc1089c94f',
  'attestationName': 'studentInstituteAttest',
  'closed': True},
 {'id': '325d7a14-900e-45d8-a928-82c92461969c',
  'entity': 'Student',
  'entityId': '1-8333e64a-4f4e-4d98-95b3-c4753f47acba',
  'propertyURI': '',
  'createdAt': '2022-02-15T07:57:08.877+00:00',
  'attestedOn': None,
  'status': 'OPEN',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,

## approve the claim

In [223]:
claim_id

'96852340-a171-43b6-b357-d8666aafac60'

In [224]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}
data = {
    "action":"GRANT_CLAIM",
}
response = requests.post('http://localhost:8081/api/v1/Teacher/claims/%s/attest'%claim_id, headers=headers, json=data)
print(response.status_code)
response.json()

200


{'resmsgid': '',
 'msgid': '4abdf275-4259-4d89-a40c-661e85bdabb8',
 'err': '',
 'status': 'SUCCESSFUL',
 'errmsg': ''}

In [225]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/Teacher/claims', headers=headers)
print(response.status_code)
claim_id = response.json()[0]["id"]
response.json()

200


[{'id': '7e912173-329a-490f-9561-c9943ce6f693',
  'entity': 'Student',
  'entityId': '1-383ecafc-eea0-4bcc-a061-42336753ae81',
  'propertyURI': '',
  'createdAt': '2022-02-15T12:54:51.731+00:00',
  'attestedOn': '2022-02-15T12:57:40.834+00:00',
  'status': 'CLOSED',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,
  'propertyData': '{"name":"Test 81540432738","institute":"Some other school"}',
  'attestationId': '1-524c50f7-7241-4c7d-966c-a4dc1089c94f',
  'attestationName': 'studentInstituteAttest',
  'closed': True},
 {'id': '325d7a14-900e-45d8-a928-82c92461969c',
  'entity': 'Student',
  'entityId': '1-8333e64a-4f4e-4d98-95b3-c4753f47acba',
  'propertyURI': '',
  'createdAt': '2022-02-15T07:57:08.877+00:00',
  'attestedOn': None,
  'status': 'OPEN',
  'conditions': "(ATTESTOR#$.experience.[*].institute#.contains('Some other school'))",
  'attestorEntity': 'Teacher',
  'requestorName': None,

In [226]:
headers = {
    'content-type': 'application/json',
    'Authorization': 'Bearer %s'%token, 
}

response = requests.get('http://localhost:8081/api/v1/%s/%s'%("Student", sid), headers=headers)
print(response.status_code)
response.json()

200


{'educationDetails': [{'institute': 'Some other school',
   'osid': '1-45cecd72-ded5-4540-b3c8-861628537143'}],
 'studentInstituteAttest': [{'additionalInput': {'osid': '1-f8f46a25-8628-4e7e-94fa-2737018ea046',
    'idx': '1-f487ba74-2b5b-4d4b-9619-471e75aec617'},
   'entityName': 'Student',
   '_osState': 'PUBLISHED',
   'name': 'studentInstituteAttest',
   'entityId': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
   '_osAttestedData': '{"name":"Test 97767099632","institute":"Some other school"}',
   'osid': '1-12c772ad-6dcc-412b-b61c-318ded5a0961',
   '_osClaimId': '96852340-a171-43b6-b357-d8666aafac60',
   'propertyData': '{"name":"Test 97767099632","institute":"Some other school"}'}],
 'osid': '1-f487ba74-2b5b-4d4b-9619-471e75aec617',
 'identityDetails': {'osid': '1-ea45774c-cd00-4bab-8c82-f0bf361c4990',
  'gender': 'Male',
  'fullName': 'Test 97767099632'}}